# 지역기준 데이터셋 구성 (수원) v.0.1

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta, time 
import json

with open("./db_config.json", "r") as f:
    config = json.load(f)
    
DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_PORT = config["DB_PORT"]

# SQLAlchemy 엔진 생성
engine_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(engine_url)
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
except Exception as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit()

MySQL 데이터베이스에 성공적으로 연결되었습니다.


In [4]:
# --- 데이터베이스에서 데이터 로드하는 함수 ---
def load_table_to_df(table_name, engine):
    """지정된 테이블에서 모든 데이터를 Pandas DataFrame으로 로드합니다."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        print(f"'{table_name}' 테이블 로드 완료. {len(df)} 행.")
        # 날짜/시간 컬럼 타입 변환 (필요시)
        if 'game_date' in df.columns:
            df['game_date'] = pd.to_datetime(df['game_date'])
        if 'accident_date' in df.columns:
            df['accident_date'] = pd.to_datetime(df['accident_date'])
        if 'weather_date' in df.columns:
            df['weather_date'] = pd.to_datetime(df['weather_date'])
        # start_time, end_time, weather_time은 문자열로 로드될 수 있으므로, 필요시 time 객체로 변환
        return df
    except Exception as e:
        print(f"'{table_name}' 테이블 로드 중 오류 발생: {e}")
        return pd.DataFrame() # 오류 발생 시 빈 DataFrame 반환

# --- 데이터 로드 ---
stadium_df = load_table_to_df('stadium', engine)
sports_game_df = load_table_to_df('sports_game', engine)
traffic_accident_df = load_table_to_df('traffic_accident', engine)
weather_df = load_table_to_df('weather', engine)

'stadium' 테이블 로드 완료. 60 행.
'sports_game' 테이블 로드 완료. 3631 행.
'traffic_accident' 테이블 로드 완료. 34032 행.
'weather' 테이블 로드 완료. 450811 행.


In [5]:
# 1. 경기 수원시를 기준으로 데이터 셋 생성
TARGET_REGION = "경기 수원시" # 예시: 분석 대상 지역
START_DATE = pd.to_datetime("2023-01-01") # 분석 시작일
END_DATE = pd.to_datetime("2024-12-31")   # 분석 종료일

# 2. 날짜를 기준으로 데이터프레임 생성
date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
base_df = pd.DataFrame({'date': date_range})
base_df['region_code'] = TARGET_REGION

base_df

,date,region_code
0,2023-01-01,경기 수원시
1,2023-01-02,경기 수원시
2,2023-01-03,경기 수원시
3,2023-01-04,경기 수원시
4,2023-01-05,경기 수원시
...,...,...
726,2024-12-27,경기 수원시
727,2024-12-28,경기 수원시
728,2024-12-29,경기 수원시
729,2024-12-30,경기 수원시


In [6]:
# 3. target region의 Stadium code 가져오기 
if not stadium_df.empty:
    stadiums_in_target_region = stadium_df[stadium_df['region'] == TARGET_REGION]
    stadium_codes_in_region = stadiums_in_target_region['stadium_code'].unique().tolist()
    print(f"\n{TARGET_REGION} 내 경기장 코드: {stadium_codes_in_region}")
else:
    stadium_codes_in_region = []
    print(f"\n{TARGET_REGION} 내 경기장 정보 없음 또는 stadium 테이블 로드 실패.")


경기 수원시 내 경기장 코드: ['SU01', 'SU02', 'SU03', 'SU04', 'SU05']


In [8]:
# 4. 스포츠 데이터 처리
if not sports_game_df.empty and stadium_codes_in_region:
    games_in_region_df = sports_game_df[sports_game_df['stadium_code'].isin(stadium_codes_in_region)]
    games_in_region_df = games_in_region_df.rename(columns={'game_date': 'date'})
    #print(games_in_region_df)
    if not games_in_region_df.empty:
        game_summary_df = games_in_region_df.groupby('date').agg(
            game_count=('stadium_code', 'count'),
            sport_types_list=('sports_type', lambda x: list(set(x))),
            has_playoff_list=('match_type', lambda x: 1 if any('플레이오프' in str(mt).lower() for mt in x) else 0)
        ).reset_index()
        
        game_summary_df['sport_types'] = game_summary_df['sport_types_list'].apply(lambda x: ','.join(sorted(x)) if x else '없음')
        game_summary_df['has_playoff'] = game_summary_df['has_playoff_list']
        game_summary_df = game_summary_df[['date', 'game_count', 'sport_types', 'has_playoff']]
        base_df = pd.merge(base_df, game_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 경기 정보 없음.")
        base_df['game_count'] = 0
        base_df['sport_types'] = '없음'
        base_df['has_playoff'] = 0
else:
    print("sports_game_df 로드 실패 또는 대상 지역 내 경기장 없음.")
    base_df['game_count'] = 0
    base_df['sport_types'] = '없음'
    base_df['has_playoff'] = 0

base_df['game_count'] = base_df['game_count'].fillna(0).astype(int)
base_df['sport_types'] = base_df['sport_types'].fillna('없음')
base_df['has_playoff'] = base_df['has_playoff'].fillna(0).astype(int)

base_df

,date,region_code,game_count,sport_types,has_playoff
0,2023-01-01,경기 수원시,0,없음,0
1,2023-01-02,경기 수원시,0,없음,0
2,2023-01-03,경기 수원시,0,없음,0
3,2023-01-04,경기 수원시,1,여자배구,0
4,2023-01-05,경기 수원시,0,없음,0
...,...,...,...,...,...
726,2024-12-27,경기 수원시,0,없음,0
727,2024-12-28,경기 수원시,0,없음,0
728,2024-12-29,경기 수원시,0,없음,0
729,2024-12-30,경기 수원시,0,없음,0


In [9]:
# 5. 사고데이터 처리
if not traffic_accident_df.empty:
    accidents_in_region_df = traffic_accident_df[traffic_accident_df['region'] == TARGET_REGION]
    accidents_in_region_df = accidents_in_region_df.rename(columns={'accident_date': 'date'})
    if not accidents_in_region_df.empty:
        accident_summary_df = accidents_in_region_df.groupby('date').agg(
            accident_count_sum=('accident_count', 'sum')
        ).reset_index()
        accident_summary_df = accident_summary_df.rename(columns={'accident_count_sum': 'accident_count'})
        
        base_df = pd.merge(base_df, accident_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 교통사고 정보 없음.")
        base_df['accident_count'] = 0
else:
    print("traffic_accident_df 로드 실패.")
    base_df['accident_count'] = 0
    
base_df['accident_count'] = base_df['accident_count'].fillna(0).astype(int)

In [10]:
base_df

,date,region_code,game_count,sport_types,has_playoff,accident_count
0,2023-01-01,경기 수원시,0,없음,0,15
1,2023-01-02,경기 수원시,0,없음,0,12
2,2023-01-03,경기 수원시,0,없음,0,8
3,2023-01-04,경기 수원시,1,여자배구,0,12
4,2023-01-05,경기 수원시,0,없음,0,14
...,...,...,...,...,...,...
726,2024-12-27,경기 수원시,0,없음,0,12
727,2024-12-28,경기 수원시,0,없음,0,15
728,2024-12-29,경기 수원시,0,없음,0,14
729,2024-12-30,경기 수원시,0,없음,0,9


In [13]:
# 6. 날씨 데이터 처리
if not weather_df.empty:
    weather_region_df = weather_df[weather_df['region'] == "수원"]
    weather_region_df = weather_region_df.rename(columns={'weather_date': 'date'})

    if not weather_region_df.empty:
        # 날씨 데이터는 하루에 여러 번 기록될 수 있으므로, 일별 집계 필요
        weather_summary_df = weather_region_df.groupby('date').agg(
            temperature=('temperature', 'mean'),
            precipitation=('precipitation', 'sum'),
            snow_depth=('snow_depth', 'sum'),
            avg_cloud_amount=('cloud_amount', 'mean') # 대표 날씨 상태 추론용
        ).reset_index()

        def get_weather_condition(row):
            if (pd.isna(row['precipitation']) or pd.isna(row['snow_depth'])) and pd.isna(row['avg_cloud_amount']):
                return '정보없음' # 데이터가 없는 경우
            if row['precipitation'] > 0 or pd.isna(row['snow_depth']) > 0:
                return '비/눈'
            elif pd.notna(row['avg_cloud_amount']):
                if row['avg_cloud_amount'] >= 7: # (0-10 기준)
                    return '흐림'
                elif row['avg_cloud_amount'] >= 3:
                    return '구름조금' # 또는 '약간흐림' 등
                else:
                    return '맑음'
            return '정보없음' # 강수량 없고 구름 정보도 없는 경우
            
        weather_summary_df['weather_condition'] = weather_summary_df.apply(get_weather_condition, axis=1)
        weather_summary_df = weather_summary_df[['date', 'temperature', 'precipitation', 'snow_depth', 'weather_condition']]
        
        base_df = pd.merge(base_df, weather_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 날씨 정보 없음.")
        base_df['temperature'] = pd.NA
        base_df['precipitation'] = pd.NA
        base_df['snow_depth'] = pd.NA
        base_df['weather_condition'] = '정보없음'
else:
    print("weather_df 로드 실패.")
    base_df['temperature'] = pd.NA
    base_df['precipitation'] = pd.NA
    base_df['snow_depth'] = pd.NA
    base_df['weather_condition'] = '정보없음'

In [14]:
base_df

,date,region_code,game_count,sport_types,has_playoff,accident_count,temperature_x,precipitation_x,snow_depth_x,weather_condition_x,temperature_y,precipitation_y,snow_depth_y,weather_condition_y,temperature,precipitation,snow_depth,weather_condition
0,2023-01-01,경기 수원시,0,없음,0,15,-1.979167,0.0,0.0,맑음,-1.979167,0.0,0.0,맑음,-1.979167,0.0,0.0,맑음
1,2023-01-02,경기 수원시,0,없음,0,12,-5.250000,0.0,0.0,맑음,-5.250000,0.0,0.0,맑음,-5.250000,0.0,0.0,맑음
2,2023-01-03,경기 수원시,0,없음,0,8,-6.245833,0.0,0.0,맑음,-6.245833,0.0,0.0,맑음,-6.245833,0.0,0.0,맑음
3,2023-01-04,경기 수원시,1,여자배구,0,12,-3.458333,0.0,0.0,맑음,-3.458333,0.0,0.0,맑음,-3.458333,0.0,0.0,맑음
4,2023-01-05,경기 수원시,0,없음,0,14,-3.629167,0.0,0.0,구름조금,-3.629167,0.0,0.0,구름조금,-3.629167,0.0,0.0,구름조금
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2024-12-27,경기 수원시,0,없음,0,12,-2.679167,0.0,0.6,맑음,-2.679167,0.0,0.6,맑음,-2.679167,0.0,0.6,맑음
727,2024-12-28,경기 수원시,0,없음,0,15,-3.137500,0.0,0.9,맑음,-3.137500,0.0,0.9,맑음,-3.137500,0.0,0.9,맑음
728,2024-12-29,경기 수원시,0,없음,0,14,0.475000,0.0,0.0,구름조금,0.475000,0.0,0.0,구름조금,0.475000,0.0,0.0,구름조금
729,2024-12-30,경기 수원시,0,없음,0,9,4.958333,0.0,0.0,구름조금,4.958333,0.0,0.0,구름조금,4.958333,0.0,0.0,구름조금


In [15]:
%pip install holidays

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 10.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
# 7. 휴일처리
weekday_map_kr = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
base_df['weekday'] = base_df['date'].dt.dayofweek.map(weekday_map_kr)

try:
    import holidays
    # base_df['date']에서 연도를 뽑아와서 unique 값으로 추출
    kr_holidays = holidays.KR(years=base_df['date'].dt.year.unique().tolist()) 
    is_statutory_holiday = base_df['date'].apply(lambda d: d in kr_holidays)
    is_saturday = (base_df['weekday'] == '토')
    is_sunday = (base_df['weekday'] == '일')
    # 3. 세 가지 조건을 OR 연산자로 결합하여 is_holiday 컬럼 생성
    # (하나라도 True이면 True -> 1, 모두 False이면 False -> 0)
    base_df['is_holiday'] = (is_statutory_holiday | is_saturday | is_sunday).astype(int)
except ImportError:
    print("`holidays` 라이브러리가 설치되지 않았습니다. `pip install holidays`로 설치해주세요. 'is_holiday'는 0으로 처리됩니다.")
    base_df['is_holiday'] = 0
except Exception as e:
    print(f"공휴일 정보 처리 중 오류: {e}. 'is_holiday'는 0으로 처리됩니다.")
    base_df['is_holiday'] = 0

In [17]:
# 8. Finalize
final_df = base_df[[
    'date', 'region_code', 'accident_count', 'game_count', 'sport_types',
    'has_playoff', 'temperature', 'precipitation', 'snow_depth', 'weather_condition',
    'is_holiday', 'weekday'
]].copy() # SettingWithCopyWarning 방지를 위해 .copy() 사용

final_df['date'] = final_df['date'].dt.strftime('%Y-%m-%d')
final_df['temperature'] = pd.to_numeric(final_df['temperature'], errors='coerce').round(1)
final_df['precipitation'] = pd.to_numeric(final_df['precipitation'], errors='coerce').round(1)

# --- 활용 가능 분석 예시 ---
print("\n--- 활용 가능 분석 예시 ---")
if not final_df.empty:
    avg_accidents_game_day = final_df[final_df['game_count'] > 0]['accident_count'].mean()
    avg_accidents_no_game_day = final_df[final_df['game_count'] == 0]['accident_count'].mean()
    if pd.notna(avg_accidents_game_day):
        print(f"평균 사고 건수 (경기 있는 날): {avg_accidents_game_day:.2f}") 
    else :
        print(f"평균 사고 건수 (경기 있는 날): N/A") 
    if pd.notna(avg_accidents_no_game_day):   
        print(f"평균 사고 건수 (경기 없는 날): {avg_accidents_no_game_day:.2f}")
    else :
        print(f"평균 사고 건수 (경기 없는 날): N/A")
    playoff_days_df = final_df[(final_df['game_count'] > 0) & (final_df['has_playoff'] == 1)]
    if not playoff_days_df.empty:
        avg_accidents_playoff_day = playoff_days_df['accident_count'].mean()
        if pd.notna(avg_accidents_playoff_day):
            print(f"평균 사고 건수 (플레이오프 경기 있는 날): {avg_accidents_playoff_day:.2f}")
        else:
            print("평균 사고 건수 (플레이오프 경기 있는 날): N/A")
    else:
        print("분석 기간/지역 내 플레이오프 경기 데이터 없음.")
else:
    print("생성된 최종 데이터셋이 비어있어 분석을 수행할 수 없습니다.")

# 데이터베이스 엔진 연결 종료 (스크립트가 끝나면 자동으로 닫히지만 명시적으로 호출 가능)
if 'engine' in locals() and engine:
    engine.dispose()
    print("\n데이터베이스 연결이 종료되었습니다.")


--- 활용 가능 분석 예시 ---
평균 사고 건수 (경기 있는 날): 12.52
평균 사고 건수 (경기 없는 날): 13.29
평균 사고 건수 (플레이오프 경기 있는 날): 12.83

데이터베이스 연결이 종료되었습니다.


In [18]:
print("\n--- Final Dataset ---")
final_df


--- Final Dataset ---


,date,region_code,accident_count,game_count,sport_types,has_playoff,temperature,precipitation,snow_depth,weather_condition,is_holiday,weekday
0,2023-01-01,경기 수원시,15,0,없음,0,-2.0,0.0,0.0,맑음,1,일
1,2023-01-02,경기 수원시,12,0,없음,0,-5.2,0.0,0.0,맑음,0,월
2,2023-01-03,경기 수원시,8,0,없음,0,-6.2,0.0,0.0,맑음,0,화
3,2023-01-04,경기 수원시,12,1,여자배구,0,-3.5,0.0,0.0,맑음,0,수
4,2023-01-05,경기 수원시,14,0,없음,0,-3.6,0.0,0.0,구름조금,0,목
...,...,...,...,...,...,...,...,...,...,...,...,...
726,2024-12-27,경기 수원시,12,0,없음,0,-2.7,0.0,0.6,맑음,0,금
727,2024-12-28,경기 수원시,15,0,없음,0,-3.1,0.0,0.9,맑음,1,토
728,2024-12-29,경기 수원시,14,0,없음,0,0.5,0.0,0.0,구름조금,1,일
729,2024-12-30,경기 수원시,9,0,없음,0,5.0,0.0,0.0,구름조금,0,월


In [19]:
from datetime import datetime

final_df.to_csv(f"2nd-dataset_{TARGET_REGION}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv", index=False)

In [20]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               731 non-null    object 
 1   region_code        731 non-null    object 
 2   accident_count     731 non-null    int64  
 3   game_count         731 non-null    int64  
 4   sport_types        731 non-null    object 
 5   has_playoff        731 non-null    int64  
 6   temperature        731 non-null    float64
 7   precipitation      731 non-null    float64
 8   snow_depth         731 non-null    float64
 9   weather_condition  731 non-null    object 
 10  is_holiday         731 non-null    int64  
 11  weekday            731 non-null    object 
dtypes: float64(3), int64(4), object(5)
memory usage: 68.7+ KB
